In [1]:
import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm

from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.logger import configure

# ========= 1. Environment Wrapper =========
class EpisodicRewardWrapper(gym.RewardWrapper):
    def __init__(self, env):
        super().__init__(env)
        self.cumulative_reward = 0.0

    def step(self, action):
        result = self.env.step(action)
        if len(result) == 5:
            obs, reward, terminated, truncated, info = result
            done = terminated or truncated
        else:
            obs, reward, done, info = result

        # 记录累计的reward
        self.cumulative_reward += reward
        
        if done:
            reward = self.cumulative_reward
            self.cumulative_reward = 0.0
        else:
            reward = 0.0
        
        return obs, reward, done, info

    def reset(self, **kwargs):
        self.cumulative_reward = 0.0
        result = self.env.reset(**kwargs)
        if isinstance(result, tuple):
            obs, _ = result
        else:
            obs = result
        return obs




# ========= 2. 轨迹存储 =========
class TrajectoryReplay:
    def __init__(self):
        self.trajectories = []

    def add_trajectory(self, traj):
        self.trajectories.append(traj)

    def sample(self, batch_size):
        indices = np.random.randint(0, len(self.trajectories), size=batch_size)
        return [self.trajectories[i] for i in indices]

    def __len__(self):
        return len(self.trajectories)



# ========= 3. Reward Proxy ==========
class RewardModel(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_size=256):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(state_dim + action_dim, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
        )
        self.mu_layer = nn.Linear(hidden_size, 1)
        self.log_sigma_layer = nn.Linear(hidden_size, 1)

    def forward(self, s, a):
        x = torch.cat([s, a], dim=-1)
        x = self.net(x)
        mu = self.mu_layer(x)
        log_sigma = self.log_sigma_layer(x)
        sigma = torch.clamp(torch.exp(log_sigma), min=1e-3, max=2.0)
        return mu, sigma


# ========= 4. Train a reward model with batch trajectories ==========
def train_reward_model_gaussian_loo(reward_model, optimizer, trajectories, device='cpu'):
    """
    使用留一法训练奖励模型：
    对于每条轨迹 τ = { (s₀, a₀), ..., (s_{T-1}, a_{T-1}) }，
      - 真实轨迹奖励 R_ep = ∑ₜ rₜ（假设中间步奖励为0，只有最后一步累计）
      - 对每个时间步 i：
            * 得到每个 (s_t, a_t) 的预测参数 μ_t 和 σ_t；
            * 对于 t ≠ i，单独采样噪声 ε_{i,t} ~ N(0,1)，计算采样奖励：
                  r(s_t,a_t) = μ_t + σ_t * ε_{i,t}
            * 计算目标值 target_i = R_ep - ∑_{t ≠ i} r(s_t,a_t)
            * 计算该时间步的负对数似然：
                  nll_i = log(σ_i) + ((target_i - μ_i)²) / (2σ_i²)
    最后对每条轨迹所有时间步的 nll 取平均，再对所有轨迹取平均作为最终损失。
    """
    reward_model.train()
    total_loss = 0.0
    total_episodes = 0

    for traj in trajectories:
        # 真实轨迹奖励（假设中间步奖励为0，只有最后一步累计）
        R_ep = sum([t[2] for t in traj])
        Tj = len(traj)
        if Tj == 0:
            continue

        # 收集状态和动作
        s_list, a_list = [], []
        for (s, a, r, s_next, done) in traj:
            s_list.append(s)
            a_list.append(a)
        states = torch.tensor(s_list, dtype=torch.float, device=device)
        actions = torch.tensor(a_list, dtype=torch.float, device=device)
        
        # 预测整个轨迹每个 (s,a) 的参数，输出 shape 为 [Tj, 1]
        mu, sigma = reward_model(states, actions)
        
        nll_list = []
        for i in range(Tj):
            # 构造一个 mask，排除第 i 个时间步
            mask = torch.ones(Tj, dtype=torch.bool, device=device)
            mask[i] = False
            
            # 对于每个 t ≠ i，单独采样噪声 ε_{i,t} ~ N(0,1)
            epsilons = torch.randn((mask.sum(), 1), device=device)
            
            # 计算 t ≠ i 的采样奖励： r = μ + σ * ε_{i,t}
            r_samples = mu[mask] + epsilons * sigma[mask]
            leave_one_sum = r_samples.sum()  # 除去第 i 步的总奖励预测
            
            # 第 i 步的目标：R_ep - (其它时间步的采样奖励之和)
            target_i = R_ep - leave_one_sum
            
            # 计算第 i 步的负对数似然损失
            nll_i = torch.log(sigma[i]) + ((target_i - mu[i])**2) / (2 * sigma[i]**2)
            nll_list.append(nll_i)
        
        loss_traj = torch.stack(nll_list).mean()  # 对单条轨迹所有时刻取平均
        total_loss += loss_traj
        total_episodes += 1

    if total_episodes > 0:
        loss_mean = total_loss / total_episodes
    else:
        loss_mean = torch.tensor(0.0, device=device)

    optimizer.zero_grad()
    loss_mean.backward()
    optimizer.step()
    return loss_mean.item()








# ========= 5. How to collect a trajectory ==========
def collect_episodes(env, model, n_episodes, device='cpu'):
    trajectories = []
    for _ in range(n_episodes):
        result = env.reset()
        if isinstance(result, tuple):
            obs, _ = result
        else:
            obs = result
        done = False
        traj = []
        while not done:
            if obs is None or (hasattr(obs, '__len__') and len(obs) == 0):
                raise ValueError("Observation is empty, check env.reset() output")
            obs_tensor = torch.tensor(obs, dtype=torch.float, device=device).unsqueeze(0)
            with torch.no_grad():
                action, _states = model.predict(obs_tensor.cpu().numpy(), deterministic=False)
            result = env.step(action[0])
            
            if len(result) == 5:
                next_obs, reward, terminated, truncated, info = result
                done = terminated or truncated
            
            else:
                next_obs, reward, done, info = result
            traj.append((obs, action[0], reward, next_obs, done))
            obs = next_obs
        trajectories.append(traj)
    return trajectories


# ========= 6. 将新的reward模型生成的代理奖励存入SAC的ReplayBuffer ==========
def add_shaped_transitions_to_replay(model, reward_model, trajectories, device='cpu'):
    for traj in trajectories:
        states = []
        actions = []
        next_states = []
        dones = []
        for (s, a, r, s_next, d) in traj:
            states.append(s)
            actions.append(a)
            next_states.append(s_next)
            dones.append(d)
        
        s_tensor = torch.tensor(states, dtype=torch.float, device=device)
        a_tensor = torch.tensor(actions, dtype=torch.float, device=device)
        
        with torch.no_grad():
            mu, _ = reward_model(s_tensor, a_tensor)  # 这里我们用 μ 作为代理奖励
        shaped_rewards = mu.cpu().numpy().squeeze(-1)
        
        for i in range(len(traj)):
            s, a, _, s_next, d = traj[i]
            r_shaped = shaped_rewards[i]
            model.replay_buffer.add(
                s, s_next, a, r_shaped, d, infos=[{}],
            )

# ========= 7. 主函数 ==========
def main():
    class Args:
        env = "HalfCheetah-v4"
        episodes = 2000
        steps_per_update = 1000
        # 使用GPU
        device = "cuda"
    
    args = Args()
    device = torch.device(args.device if torch.cuda.is_available() else "cpu")

    # 创建环境（注意环境包装仍然不改变总reward的逻辑）
    base_env = gym.make(args.env)
    env = EpisodicRewardWrapper(base_env)
    vec_env = DummyVecEnv([lambda: env])

    # 创建SAC模型（注意这里SAC的训练依赖于ReplayBuffer，我们将手动往buffer中填入代理奖励）
    model = SAC(
        policy="MlpPolicy",
        env=vec_env,
        verbose=1,
        seed=42,
        buffer_size=100000,
        learning_starts=0,
        train_freq=1,
        gradient_steps=0,
        batch_size=256,
        gamma=0.99,
        device=device
    )
    # 配置logger
    from stable_baselines3.common.logger import configure
    model._logger = configure(folder=None, format_strings=["stdout"])

    # 创建奖励模型（现在输出μ和σ）
    state_dim = base_env.observation_space.shape[0]
    action_dim = base_env.action_space.shape[0]
    reward_model = RewardModel(state_dim, action_dim, hidden_size=256).to(device)
    reward_optimizer = optim.Adam(reward_model.parameters(), lr=3e-4)

    traj_replay = TrajectoryReplay()

    ep_record = []
    for ep in tqdm(range(args.episodes)):
        # (a) 收集轨迹
        new_trajectories = collect_episodes(env, model, n_episodes=1, device=device)
        for traj in new_trajectories:
            traj_replay.add_trajectory(traj)
        
        # (b) 当有足够轨迹后，用Gaussian对数似然训练奖励模型
        if len(traj_replay) >= 8:
            batch_size_trajectories = 8
            sampled_trajs = traj_replay.sample(batch_size_trajectories)
            loss_val = train_reward_model_gaussian_loo(
                reward_model, reward_optimizer, sampled_trajs, device=device
            )
        
        # (c) 将新轨迹用更新后的奖励模型生成的代理奖励存入SAC的ReplayBuffer
        add_shaped_transitions_to_replay(model, reward_model, new_trajectories, device=device)
        
        # (d) 用SAC更新策略
        if len(traj_replay) >= 8:
            model.train(args.steps_per_update)
        
        # (e) 简单评估：输出最新轨迹的真实环境回报
        ep_return = sum([t[2] for t in new_trajectories[-1]])
        ep_record.append(ep_return)
        if ep % 10 == 0:
            print(f"Episode {ep+1}/{args.episodes}, real env return = {ep_return:.2f}")

    print("Training finished.")
    return ep_record

In [2]:
train1 = main()

Using cuda device


  0%|                                          | 1/2000 [00:00<11:41,  2.85it/s]

Episode 1/2000, real env return = -209.21


  1%|▏                                      | 11/2000 [00:36<4:11:22,  7.58s/it]

Episode 11/2000, real env return = -309.77


...

Episode 1981/2000, real env return = 9276.13


100%|████████████████████████████████████▊| 1991/2000 [4:46:55<00:52,  5.86s/it]

Episode 1991/2000, real env return = 8791.17


100%|█████████████████████████████████████| 2000/2000 [4:47:47<00:00,  8.63s/it]


Training finished.


In [3]:
train2 = main()

Using cuda device


  0%|                                          | 1/2000 [00:00<10:42,  3.11it/s]

Episode 1/2000, real env return = -231.54


  1%|▏                                      | 11/2000 [00:44<4:59:11,  9.03s/it]

Episode 11/2000, real env return = -329.42


...

Episode 1981/2000, real env return = 3414.42


100%|████████████████████████████████████▊| 1991/2000 [4:48:14<00:52,  5.84s/it]

Episode 1991/2000, real env return = 3194.33


100%|█████████████████████████████████████| 2000/2000 [4:49:07<00:00,  8.67s/it]


Training finished.


In [4]:
train3 = main()

Using cuda device


  0%|                                          | 1/2000 [00:00<26:21,  1.26it/s]

Episode 1/2000, real env return = -164.13


  1%|▏                                      | 11/2000 [00:56<5:36:34, 10.15s/it]

Episode 11/2000, real env return = -400.90


...

Episode 1981/2000, real env return = 5544.44


100%|████████████████████████████████████▊| 1991/2000 [4:46:49<00:51,  5.73s/it]

Episode 1991/2000, real env return = 5520.68


100%|█████████████████████████████████████| 2000/2000 [4:47:41<00:00,  8.63s/it]


Training finished.


In [9]:
train4 = main()

Using cuda device


  0%|                                          | 1/2000 [00:00<11:06,  3.00it/s]

Episode 1/2000, real env return = -355.47


  1%|▏                                      | 11/2000 [00:29<2:54:20,  5.26s/it]

Episode 11/2000, real env return = -279.60


...

Episode 1981/2000, real env return = 4661.94


100%|████████████████████████████████████▊| 1991/2000 [6:10:14<01:56, 12.97s/it]

Episode 1991/2000, real env return = 4670.08


100%|█████████████████████████████████████| 2000/2000 [6:12:11<00:00, 11.17s/it]


Training finished.


In [11]:
train5 = main()

Using cuda device


  0%|                                          | 1/2000 [00:01<38:57,  1.17s/it]

Episode 1/2000, real env return = -293.40


  1%|▏                                      | 11/2000 [01:23<8:14:43, 14.92s/it]

Episode 11/2000, real env return = -395.94


...

Episode 1981/2000, real env return = 4110.41


100%|████████████████████████████████████▊| 1991/2000 [8:14:07<01:49, 12.15s/it]

Episode 1991/2000, real env return = 4638.06


100%|█████████████████████████████████████| 2000/2000 [8:15:55<00:00, 14.88s/it]


Training finished.
